# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:

- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:

- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are:

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:

Your project should not be a mere code copy-paste from somewhere. You must provide:

- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:

- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.


## **Justification**

Should be context sensitive

- Dataset (Consists of original and fixed sentece pairs)
- Weights - ...


In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import datasets
from rich import print

In [2]:
# bigram_dataset = datasets.Dataset.from_csv(
#     "bigrams.txt", sep="\t", encoding="latin-1", names=["freq", "w1", "w2"]
# )
# print(bigram_dataset)
# print(bigram_dataset[0])

In [3]:
# fivegram_dataset = datasets.Dataset.from_csv(
#     "fivegrams.txt",
#     sep="\t",
#     encoding="utf-8",
#     names=["freq", "w1", "w2", "w3", "w4", "w5"],
# )
# print(fivegram_dataset)
# print(fivegram_dataset[0])

In [5]:
# Predefined keyboard adjacency for QWERTY layout
keyboard_adjacency = {
    "q": ["w", "a", "s"],
    "w": ["q", "e", "a", "s", "d"],
    "e": ["w", "r", "s", "d", "f"],
    "r": ["e", "t", "d", "f", "g"],
    "t": ["r", "y", "f", "g", "h"],
    "y": ["t", "u", "g", "h", "j"],
    "u": ["y", "i", "h", "j", "k"],
    "i": ["u", "o", "j", "k", "l"],
    "o": ["i", "p", "k", "l"],
    "p": ["o", "l"],
    "a": ["q", "w", "s", "z", "x"],
    "s": ["a", "w", "e", "d", "z", "x", "c"],
    "d": ["s", "e", "r", "f", "x", "c", "v"],
    "f": ["d", "r", "t", "g", "c", "v", "b"],
    "g": ["f", "t", "y", "h", "v", "b", "n"],
    "h": ["g", "y", "u", "j", "b", "n", "m"],
    "j": ["h", "u", "i", "k", "n", "m"],
    "k": ["j", "i", "o", "l", "m"],
    "l": ["k", "o", "p"],
    "z": ["a", "s", "x"],
    "x": ["z", "s", "d", "c"],
    "c": ["x", "d", "f", "v"],
    "v": ["c", "f", "g", "b"],
    "b": ["v", "g", "h", "n"],
    "n": ["b", "h", "j", "m"],
    "m": ["n", "j", "k"],
}

In [3]:
def generate_candidates(word):
    letters = "abcdefghijklmnopqrstuvwxyz"
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    substitutes_adj = []
    for L, R in splits:
        if R:
            current_char = R[0].lower()
            for c in keyboard_adjacency.get(current_char, []):
                substitutes_adj.append(L + c + R[1:])

    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    substitutes = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]

    candidates = set(deletes + transposes + substitutes_adj + substitutes + inserts)
    candidates.add(word)
    return candidates

In [6]:
candidates = generate_candidates("dking")
len(candidates), list(candidates)[:10]

(286,
 ['dksng',
  'dkeing',
  'bking',
  'djking',
  'dkingk',
  'dkingl',
  'lking',
  'tdking',
  'dkingz',
  'qking'])

In [7]:
from collections import defaultdict


class SpellCorrector:
    def __init__(self, ngrams: list[int] = [], w1: float = 1.0, w2: float = 1.0):
        # indexes
        self.unigram_counts = defaultdict(int)
        self.bigram_counts = defaultdict(int)
        self.fivegram_counts = defaultdict(int)
        self.load_counts()

        # ...
        self.ngrams = ngrams if ngrams else [1, 2]
        self.VOCAB = set(self.unigram_counts.keys())
        self.VOCAB_SIZE = len(self.VOCAB)
        self.w1 = w1
        self.w2 = w2

    def load_counts(self):
        with open("unigram.csv", "r") as f:
            f.readline()  # read header line
            for line in f:
                parts = line.strip().split(",")
                if len(parts) == 2:  # frequency word1 word2
                    w, freq = parts
                    self.unigram_counts[w] = int(freq)

        with open("bigrams.txt", "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 3:  # frequency word1 word2
                    freq, w1, w2 = parts
                    self.bigram_counts[(w1, w2)] = int(freq)

        with open("fivegrams.txt", "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 6:  # frequency word1 word2 word3 word4 word5
                    freq, w1, w2, w3, w4, w5 = parts
                    self.fivegram_counts[(w1, w2, w3, w4, w5)] = int(freq)

    def unigram_prob(self, w) -> float:
        return self.unigram_counts.get(w, 0) / self.VOCAB_SIZE

    def bigram_prob(self, words, candidate_index, candidates) -> float:
        assert len(words) == 2

        def key(words, i, candidate) -> tuple:
            """
            Function to put candidate in local window of words on i'th index.
            Returns key for bigram counts.
            """
            return tuple(words[:i] + [candidate] + words[i + 1 :])

        count_candidate = self.bigram_counts.get(
            key(words, candidate_index, words[candidate_index]), 0
        )
        if count_candidate == 0:
            return 0.0

        count_candidates = sum(
            [
                self.bigram_counts.get(key(words, candidate_index, candidate), 0)
                for candidate in candidates
            ]
        )
        if count_candidates == 0:
            return 0.0

        return count_candidate / count_candidates

    def correct_word(self, index: int, window: list[str]) -> str:
        """Correct the word on index'th position in the window of the current sentence.

        Args:
            index (int): The position of the word to correct in the window.
            window (list[str]): A list of words representing the current sentence.

        Returns:
            str: The corrected word.

        Example:
            index = 2
            window = ["the", "quick", "brown", "fox"]
        """
        if index >= len(window):
            raise IndexError

        if len(self.ngrams) == 0 or any([ngram < 1 for ngram in self.ngrams]):
            raise ValueError

        word = window[index]
        if word in self.VOCAB:  # if exists, correction is not needed
            # print(f"{word} is in vocab")
            return word

        # only existing candidates
        candidates = [
            candidate
            for candidate in generate_candidates(word)
            if candidate in self.VOCAB
        ]

        # extract all previous and following words to the current word
        prevs = window[:index]
        nexts = window[index + 1 :]

        # rank based on most suitness based on stats
        ranks = []
        for candidate in candidates:
            rank = 0.0

            if 1 in self.ngrams:
                rank += self.w1 * self.unigram_prob(candidate)

            if 2 in self.ngrams:
                if len(prevs) > 0:
                    words = [prevs[-1], candidate]
                    rank += self.w2 * self.bigram_prob(words, 1, candidates)
                if len(nexts) > 0:
                    words = [candidate, nexts[0]]
                    rank += self.w2 * self.bigram_prob(words, 0, candidates)

            ranks.append((rank, candidate))

        # argmax rank
        ranks.sort(key=lambda t: t[0])

        # in case no candidates exist assume the word is that unique such should remain the same
        if len(ranks) == 0:
            return word

        return ranks[-1][1]  # word of the candidate with the highest rank

    def correct_sentence(self, sentence: str) -> str:
        sentence = sentence.lower().strip()
        words = sentence.split()
        corrected_sentence = []

        for i, word in enumerate(words):
            max_ngram = max(self.ngrams)

            # boundy indexes of local window in global sentence indexes
            left_i = max(0, i - max_ngram + 1)
            right_i = min(len(words) - 1, i + max_ngram - 1)

            # arguments to word correction
            window = words[left_i : right_i + 1]
            index = window.index(word)  # TODO: really bad

            # print(word, window, left_i, right_i + 1, i)
            # continue
            # index is local for window
            corrected_word = self.correct_word(index, window)
            corrected_sentence.append(corrected_word)

        return " ".join(corrected_sentence)

In [8]:
spell_corrector = SpellCorrector()

In [9]:
# spell_corrector.correct_word(1, ["out", "oe"], [1, 2])  # oe is suddenly in unigram index
# spell_corrector.correct_word(0, ["upeaker", "four"], [1, 2])

In [9]:
# spell_corrector.bigram_counts.get(("dying", "species"), 0)
spell_corrector.unigram_counts.get("dying", 0)

9123557

In [10]:
# Example 1: "dking sport" → "doing sport"
print(spell_corrector.correct_sentence("dking sport"))  # Output: doing sport

# ! there are litteraly no examples of "dying species" bigram in provided dataset
# Example 2: "dking species" → "dying species"
print(spell_corrector.correct_sentence("dking species"))  # Output: dying species

doing sport

doing species

## Evaluate on a test set

[Dataset](https://huggingface.co/datasets/vishnun/SpellGram)

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.


In [11]:
dataset = datasets.load_dataset("vishnun/SpellGram")["train"]
print(dataset)
print(dataset[0])

Dataset({
    features: ['source', 'target'],
    num_rows: 40000
})

{'source': 'rate the silent upeaker four out oe 6', 'target': 'rate the silent speaker four out of 6'}

In [12]:
spell_corrector = SpellCorrector()

In [13]:
from difflib import Differ
from rich.console import Console
from rich.text import Text


def highlight_differences(str1, str2):
    """Compare two strings and return a Text object with differences highlighted"""
    words1 = str1.split()
    words2 = str2.split()
    result = Text()

    # Use difflib to find differences
    diffs = list(Differ().compare(words1, words2))

    for diff in diffs:
        if diff.startswith("  "):  # unchanged
            result.append(diff[2:] + " ", style="white")
        elif diff.startswith("- "):  # removed
            result.append(diff[2:] + " ", style="red")
        elif diff.startswith("+ "):  # added
            result.append(diff[2:] + " ", style="green")

    return result

In [14]:
import Levenshtein

console = Console()

for row in [dataset[0], dataset[100]]:
    source = row["source"]
    target = row["target"]

    corrected = spell_corrector.correct_sentence(source)
    distance = Levenshtein.distance(corrected, target)

    console.print("\n[bold]Source:[/bold]", source)
    console.print("[bold]Corrected vs Target:[/bold]")
    console.print(highlight_differences(corrected, target))
    console.print(f"Levenstein distance: [bold]{distance}[/bold]")
    console.print("─" * 50)  # Unicode box drawing character for cleaner separator

Source: rate the silent upeaker four out oe 6

Corrected vs Target:

rate the silent speaker four out oe a of 6 

Levenstein distance: 2

──────────────────────────────────────────────────

Source: it was done pretty qjickly in about five dkys or something

Corrected vs Target:

it was done pretty quickly in about five days or something 

Levenstein distance: 0

──────────────────────────────────────────────────

#### Calculate statistics


In [15]:
def calculate_metrics(ngrams, w1, w2):
    distance_with_correction = 0.0
    distance_winout_correction = 0.0

    spell_corrector = SpellCorrector(ngrams=ngrams, w1=w1, w2=w2)

    for row in dataset:
        source = row["source"]
        target = row["target"]

        if target is None:
            target = ""

        # obtain correcion from spell checker
        corrected = spell_corrector.correct_sentence(source)

        # accumulate "error" levenstein distance for both cases (with correction and without)
        distance_with_correction += Levenshtein.distance(corrected, target)
        distance_winout_correction += Levenshtein.distance(source, target)

    corrected = distance_with_correction / len(dataset)
    correctedless = distance_winout_correction / len(dataset)

    return {
        "metric": 1 - corrected / correctedless,
        "corrected": corrected,
        "correctedless": correctedless,
    }

In [16]:
metrics = calculate_metrics([1, 2], 1.0, 1.0)
print(metrics)

{'metric': 0.25545545545545545, 'corrected': 1.952475, 'correctedless': 2.622375}

In [17]:
metrics = calculate_metrics([1, 2], 1.0, 10.0)
print(metrics)

{'metric': 0.25729539062872386, 'corrected': 1.94765, 'correctedless': 2.622375}

In [18]:
metrics = calculate_metrics([1], 1.0, 1.0)
print(metrics)

{'metric': 0.254997854997855, 'corrected': 1.953675, 'correctedless': 2.622375}

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)
